# Classificação faixa TRL - Bertimbau + Modelos ML

- BERTimbau Base (aka "bert-base-portuguese-cased")

## Código Versão 1.0 - 21 OUT 23

- Uso dos classificadores: *Modelos de ML*
- Matriz de confusão;
- Semente 42

- Esse código adota a simplificação proposta na dissertação, no sentido de obtermos a RV, depois alizarmos o k-fold. Dessa forma, temos uma econnomia computacional.

In [2]:
# dataset.csv   ou  dataset_pre_processado_1.csv  ou  dataset_pre_processado_stem_2.csv
#     CSV1                  CSV2                                   CSV3
dataset = "dataset.csv"

In [3]:
print("Lembre-se estamos usando o dataset: " + dataset)

Lembre-se estamos usando o dataset: dataset.csv


In [4]:
melhor_modelo = 'best_model_TRL_bertimbau_base_ml' + '.bin'
melhor_modelo

'best_model_TRL_bertimbau_base_ml.bin'

- Quantidade de tokens máxima = 242

In [5]:
MAX_LEN = 242

## Preparação

### Bibliotecas e ambiente

In [6]:
!pip install -qq transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 41.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 53.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 52.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 36.7 MB/s eta 0:00:00


In [7]:
!pip install -q -U watermark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 21.8 MB/s eta 0:00:00


In [8]:
!nvidia-smi

Thu Oct 26 11:49:55 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [9]:
import transformers
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
import torch

import numpy as np
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from collections import defaultdict
import textwrap

from tqdm import tqdm
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

%matplotlib inline
%config InlineBackend.figure_format='retina'

sns.set(style='whitegrid', palette='muted', font_scale=1.2)

HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]

sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))

rcParams['figure.figsize'] = 12, 8

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [10]:
from sklearn.model_selection import cross_val_predict, KFold
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn import naive_bayes, svm
from sklearn.naive_bayes import ComplementNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.preprocessing import MinMaxScaler

In [11]:
%reload_ext watermark
%watermark -v -p numpy,pandas,torch,transformers

Python implementation: CPython
Python version       : 3.10.12
IPython version      : 7.34.0

numpy       : 1.23.5
pandas      : 1.5.3
torch       : 2.1.0+cu118
transformers: 4.34.1



# Particoes KFOLD

- Para mais detalhes, consulte o código *1-kfold.ipynb* em que as partições foram sorteadas. As células a seguir recriam o resultado obtido.

In [12]:
array1 = np.array([  0,   1,   2,   3,   5,   7,   9,  10,  11,  12,  13,  14,  15,
         16,  17,  19,  20,  21,  22,  23,  25,  26,  27,  28,  30,  32,
         33,  34,  35,  36,  37,  39,  40,  44,  45,  46,  47,  48,  49,
         50,  52,  53,  54,  55,  56,  57,  59,  60,  61,  63,  64,  66,
         68,  69,  70,  71,  72,  75,  76,  77,  78,  79,  80,  81,  82,
         83,  84,  85,  87,  91,  92,  93,  94,  96,  97,  98,  99, 100,
        102, 103, 104, 105, 106, 108, 109, 110, 112, 113, 114, 115, 116,
        117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 129, 130,
        131, 132, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 145,
        146, 148, 150, 151, 153, 154, 155, 157, 158, 159, 160, 161, 163,
        164, 165, 166, 167])

array2 = np.array([  0,   1,   3,   4,   5,   6,   7,   8,  10,  13,  14,  15,  16,
         17,  18,  19,  20,  22,  23,  24,  25,  26,  27,  29,  30,  31,
         34,  36,  37,  38,  40,  41,  42,  43,  44,  45,  47,  49,  51,
         52,  53,  54,  55,  57,  58,  59,  60,  61,  62,  63,  64,  65,
         66,  67,  68,  69,  70,  71,  73,  74,  75,  76,  77,  80,  83,
         84,  86,  88,  89,  90,  92,  93,  94,  95,  96,  97,  98,  99,
        100, 101, 102, 104, 105, 107, 109, 110, 111, 112, 113, 114, 115,
        116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128,
        129, 130, 132, 133, 135, 136, 137, 138, 139, 142, 143, 144, 145,
        147, 149, 150, 151, 152, 153, 154, 155, 156, 157, 160, 161, 162,
        163, 165, 166, 167])
array3 = np.array([  1,   2,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,  14,
         17,  18,  20,  21,  22,  23,  24,  25,  28,  29,  30,  31,  32,
         33,  35,  36,  37,  38,  39,  41,  42,  43,  44,  45,  46,  47,
         48,  50,  51,  52,  54,  55,  56,  57,  58,  59,  62,  63,  65,
         67,  70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,
         82,  84,  85,  86,  87,  88,  89,  90,  91,  93,  95,  96,  97,
         98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 111,
        112, 113, 114, 118, 120, 121, 122, 123, 124, 126, 128, 129, 131,
        132, 133, 134, 135, 136, 138, 139, 140, 141, 142, 144, 146, 147,
        148, 149, 150, 151, 152, 153, 155, 156, 158, 159, 160, 161, 162,
        163, 164, 165, 166])
array4 = np.array([  0,   2,   3,   4,   6,   8,   9,  10,  11,  12,  15,  16,  18,
         19,  20,  21,  22,  24,  25,  26,  27,  28,  29,  31,  32,  33,
         34,  35,  37,  38,  39,  40,  41,  42,  43,  45,  46,  47,  48,
         49,  50,  51,  53,  56,  57,  58,  60,  61,  62,  63,  64,  65,
         66,  67,  68,  69,  71,  72,  73,  74,  75,  76,  78,  79,  80,
         81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,  92,  93,
         94,  95,  98, 100, 101, 103, 104, 106, 107, 108, 109, 110, 111,
        112, 114, 115, 116, 117, 119, 121, 123, 124, 125, 127, 128, 129,
        130, 131, 133, 134, 135, 137, 139, 140, 141, 142, 143, 144, 145,
        146, 147, 148, 149, 150, 152, 153, 154, 155, 156, 157, 158, 159,
        162, 163, 164, 166, 167])
array5 = np.array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  11,  12,  13,
         14,  15,  16,  17,  18,  19,  21,  23,  24,  26,  27,  28,  29,
         30,  31,  32,  33,  34,  35,  36,  38,  39,  40,  41,  42,  43,
         44,  46,  48,  49,  50,  51,  52,  53,  54,  55,  56,  58,  59,
         60,  61,  62,  64,  65,  66,  67,  68,  69,  70,  72,  73,  74,
         77,  78,  79,  81,  82,  83,  85,  86,  87,  88,  89,  90,  91,
         92,  94,  95,  96,  97,  99, 101, 102, 103, 105, 106, 107, 108,
        110, 111, 113, 115, 116, 117, 118, 119, 120, 122, 125, 126, 127,
        128, 130, 131, 132, 133, 134, 136, 137, 138, 140, 141, 143, 144,
        145, 146, 147, 148, 149, 151, 152, 154, 156, 157, 158, 159, 160,
        161, 162, 164, 165, 167])
# Criar a lista composta
kfold_train = [array1, array2, array3, array4, array5]

In [13]:
# Criar as cinco arrays numpy
array1 = np.array([  4,   6,   8,  18,  24,  29,  31,  38,  41,  42,  43,  51,  58, 62,  65,  67,  73,  74,  86,  88,  89,  90,  95, 101, 107, 111, 128, 133, 144, 147, 149, 152, 156, 162])
array2 = np.array([  2,   9,  11,  12,  21,  28,  32,  33,  35,  39,  46,  48,  50, 56,  72,  78,  79,  81,  82,  85,  87,  91, 103, 106, 108, 131, 134, 140, 141, 146, 148, 158, 159, 164])
array3 = np.array([  0,   3,  15,  16,  19,  26,  27,  34,  40,  49,  53,  60,  61, 64,  66,  68,  69,  83,  92,  94, 110, 115, 116, 117, 119, 125, 127, 130, 137, 143, 145, 154, 157, 167])
array4 = np.array([  1,   5,   7,  13,  14,  17,  23,  30,  36,  44,  52,  54,  55, 59,  70,  77,  96,  97,  99, 102, 105, 113, 118, 120, 122, 126, 132, 136, 138, 151, 160, 161, 165])
array5 = np.array([ 10,  20,  22,  25,  37,  45,  47,  57,  63,  71,  75,  76,  80,  84,  93,  98, 100, 104, 109, 112, 114, 121, 123, 124, 129, 135, 139, 142, 150, 153, 155, 163, 166])


# Criar a lista composta
kfold_test = [array1, array2, array3, array4, array5]

# Dataset

In [14]:
df = pd.read_csv(dataset)

In [15]:
novo_df = df[["resumo", "rotulo"]]

In [16]:
def adapto_faixa(faixa):
    faixa -=1
    return faixa

In [17]:
novo_df['rotulo'] = novo_df.rotulo.apply(adapto_faixa)

<ipython-input-17-d13f2641b9c4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  novo_df['rotulo'] = novo_df.rotulo.apply(adapto_faixa)


In [18]:
novo_df.tail()

,resumo,rotulo
163,Rio de Janeiro (RJ) – O Centro de Avaliações d...,2
164,Este trabalho apresenta um sistema para contro...,2
165,No contexto das comunicações táticas baseadas ...,2
166,O valor da velocidade de alvos móveis em image...,0
167,Neste artigo é apresentada a análise da seção ...,0


In [19]:
class_names = ['Faixa 1', 'Faixa 2', 'Faixa 3']

# Bertimbau

- https://huggingface.co/neuralmind/bert-base-portuguese-cased

```
@inproceedings{souza2020bertimbau,
  author    = {F{\'a}bio Souza and
               Rodrigo Nogueira and
               Roberto Lotufo},
  title     = {{BERT}imbau: pretrained {BERT} models for {B}razilian {P}ortuguese},
  booktitle = {9th Brazilian Conference on Intelligent Systems, {BRACIS}, Rio Grande do Sul, Brazil, October 20-23 (to appear)},
  year      = {2020}
}

```

In [20]:
PRE_TRAINED_MODEL_NAME = 'neuralmind/bert-base-portuguese-cased'
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

In [21]:
bert_model = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)

In [22]:
frase = 'A avaliação de prontidão tecnológica em tecnologias de interesse militar'

In [23]:
encoding = tokenizer.encode_plus(
  frase,
  max_length=20,   #Perceba que irei cortar um token da minha frase anterior
  add_special_tokens=True, # Add '[CLS]' and '[SEP]'
  return_token_type_ids=False,
  pad_to_max_length=True,
  return_attention_mask=True,
  return_tensors='pt',  # Return PyTorch tensors
)

encoding.keys()

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2606: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


dict_keys(['input_ids', 'attention_mask'])

- Informações sobre o modelo:

In [24]:
bert_model = bert_model.to(device)

In [25]:
bert_model.config.hidden_size

768

In [26]:
bert_model.config

BertConfig {
  "_name_or_path": "neuralmind/bert-base-portuguese-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.34.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 29794
}

# Aplicando o Modelo para gerar a representação vetorial

- Tutorial interessante:

```
1 - https://towardsdatascience.com/feature-extraction-with-bert-for-text-classification-533dde44dc2f
```

```
2 - https://github.com/felipemaiapolo/legalnlp/blob/main/demo/BERT/BERT_TUTORIAL.ipynb
```

*Recomendo muito o 2, pois foi feito em formato de notebook, bem redigido, comentando os passos e explicando o BERT. ALém de estar em português (material raro em termos de qualidade em nossa língua)*

### Transformando os dados em tensores:

In [27]:
# Fazendo a padronização dos textos
wrapper = textwrap.TextWrapper()
data_resumo = list(novo_df['resumo'])

- Visualizando:

In [28]:
for text in range(len(data_resumo[:2])):
  print(f'{wrapper.fill(data_resumo[text])}')
  print()

O crescente emprego de mísseis de ombro infravermelhos contra alvos
aéreos demanda a utilização de contramedidas cada vez mais modernas e
eficientes. Neste cenário, surge o Directed Infrared Countermeasure
(DIRCM), cujo objetivo é interferir no guiamento do míssil por meio de
pulsos de laser. Neste artigo, um seeker infravermelho do tipo rising
sun é modelado e simulado, sendo os efeitos da emissão de um DIRCM no
processamento do sinal avaliados. A influência de parâmetros de
frequência de repetição de pulsos e intensidade do laser são
evidenciados. Os resultados obtidos ressaltam a importância e a
necessidade do desenvolvimento de ferramentas computacionais mais
complexas, visando ao desenvolvimento da doutrina de emprego deste
tipo de contramedida.

Materiais dielétricos com baixas perdas e alta permissividade são
componentes essenciais que são utilizados em linhas de transmissões
não lineares capacitivas (LTNLs) na geração de RF. LTNLs possuem
grande potencial para gerar ondas de só

In [29]:
# Aplicando o bert_tokenizer com o comprimento máximo que definimos
encoded_inputs = tokenizer(data_resumo, padding=True, truncation=True, max_length=MAX_LEN, return_tensors="pt")


#O encoded_input está como um dicionário com 3 chaves
encoded_inputs.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

- Visualizando como o resumo ficou tokenizado, i.e., a chave ''input_ids' do encoded_inputs :

In [30]:
#Visualiando o primeiro texto após a aplicação do tokenizador
print(encoded_inputs['input_ids'][0])

print()

# Mostrando o mesmo texto decodificado
print(wrapper.fill(tokenizer.decode(encoded_inputs['input_ids'][0])))

tensor([  101,   231,  6478,  7007,   125, 16328,   125, 20462,  5771,   391,
         4020, 22281,   598, 14874, 19251,  8603,   123,  5353,   125,   598,
         8969,   591,  1078,   576,   325, 11177,   122, 10746, 22281,   119,
         3703,  5391,   117,  7872,   146,  2278,  8454, 22284,  5027,   900,
          430, 11357,   140,  2890, 14473, 22279,   113,   250, 15710, 22304,
        22311,   114,   117,  3596,  2630,   253, 12993,   307,   202, 13375,
          310,   171, 14994, 17513,   240,  1423,   125,  5995,  1409,   125,
        20909,   119,  3703,  4319,   117,   222,   176,  6505,   140,  5771,
          391,  4020,   171,  1903,  3979,   446,   233, 22285,   253, 12066,
          201,   122,  6235,   201,   117,   660,   259,  3997,   180, 11352,
          125,   222,   250, 15710, 22304, 22311,   202, 12152,   171,  4227,
        11175,   442,   119,   177,  2824,   125, 14492,   125,  5678,   125,
        21395,   125,  5995,  1409,   122,  8920,   171, 20909, 

In [31]:
# Passando os tensores para para a GPU - aceleração através da GPU
input_ids = encoded_inputs['input_ids'].to(device)
attention_mask = encoded_inputs['attention_mask'].to(device)

In [32]:
# Criando o vetor de features/caracteristicas
features = []

- Aplicando o BERT para a extração das características

In [33]:
for i in tqdm(range(len(data_resumo))):

    with torch.no_grad():
        last_hidden_states = bert_model(input_ids[i:(i+1)],attention_mask[i:(i+1)])[1].cpu().numpy().reshape(-1).tolist()
        # last_hidden_states = bert_model(input_ids[i:(i+1)])[1].cpu().numpy().reshape(-1).tolist()
    features.append(last_hidden_states)

100%|██████████| 168/168 [00:08<00:00, 20.47it/s]


In [34]:
# passando a lista features para numpy array com as features extraidas
features = np.array(features)

- Visualizando o resultado:

In [35]:
print('FEATURES: Número de linhas: ' + str(features.shape[0]) + ' Número de colunas: ', str(features.shape[1]))
print("FEATURES é um objeto " + str(type(features)))

FEATURES: Número de linhas: 168 Número de colunas:  768
FEATURES é um objeto <class 'numpy.ndarray'>


In [36]:
features[0]

array([ 3.06311809e-02,  1.49520397e-01,  1.32294416e-01, -2.43207723e-01,
       -3.00680667e-01,  1.93397328e-01,  9.97895956e-01, -2.27564096e-01,
       -1.58239052e-01, -9.89399627e-02, -9.99431074e-01,  2.17316285e-01,
        4.18448541e-03, -1.34307459e-01,  9.09331888e-02, -5.82001135e-02,
        2.44923502e-01, -8.39494914e-02,  9.99427617e-01, -7.62282848e-01,
       -1.94195777e-01, -1.48620605e-01,  2.05351729e-02,  1.76268846e-01,
       -3.32449377e-01,  8.30158666e-02,  7.46030882e-02, -1.56720486e-02,
        2.05080379e-02, -2.42098331e-01, -2.47283861e-01, -9.61611509e-01,
       -9.84005153e-01,  2.13541284e-01, -9.48252901e-02, -1.30424842e-01,
       -8.81863683e-02, -3.26770470e-02,  9.99302626e-01,  1.35206627e-02,
       -1.41007528e-01,  2.04127848e-01, -8.76255751e-01, -1.04388855e-02,
       -1.08731918e-01, -9.22489911e-02, -2.08393350e-01,  3.33597869e-01,
       -1.47161812e-01, -1.59102362e-02, -1.66461356e-02,  1.05229743e-01,
        5.19960783e-02,  

# Adaptando a representação vetorial gerada

- Perceba que até aqui, temos um numpy array chamado **features**, com 168 linhas e 768 colunas.

- Iremos transformar nosso numpy array em um dataframe novamente:

In [37]:
df_repvet = pd.DataFrame(features)
repvet_rotulado = pd.concat([df_repvet, novo_df['rotulo']], axis = 1)
repvet_rotulado.shape

(168, 769)

In [38]:
df_repvet

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,0.030631,0.149520,0.132294,-0.243208,-0.300681,0.193397,0.997896,-0.227564,-0.158239,-0.098940,...,0.216850,-0.098749,-0.158954,0.123197,-0.038104,0.143516,0.056023,-0.026564,-0.212858,0.044322
1,0.121566,0.264066,0.046265,-0.087665,-0.114877,0.057351,0.914004,-0.251580,-0.094214,-0.042298,...,0.211784,-0.007474,-0.124507,0.747115,0.076023,0.075091,0.006695,-0.047540,-0.213926,-0.121684
2,0.144653,0.043282,0.140579,-0.252127,-0.194191,0.112558,0.992621,-0.204062,-0.121806,0.061342,...,0.231817,-0.152269,-0.139368,0.432061,0.078179,0.094881,-0.054676,-0.029966,-0.069750,-0.010176
3,0.168949,0.107046,-0.021332,-0.106200,-0.136551,-0.020100,0.955896,-0.237745,-0.073313,-0.012520,...,0.293984,-0.169405,-0.054496,0.572342,0.119459,0.050222,-0.172856,-0.062752,-0.100114,-0.045655
4,-0.026465,0.219727,0.280597,-0.274787,-0.095324,0.298268,0.670012,-0.059023,-0.145148,0.173758,...,0.186416,-0.133134,-0.068988,0.866958,0.174363,0.240498,0.040937,-0.179445,-0.036021,-0.071915
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,-0.020350,-0.066606,-0.028286,0.112232,-0.086299,0.011594,0.986838,0.002186,0.138783,-0.178709,...,0.114298,-0.268188,0.071848,0.485660,-0.101025,-0.046000,-0.141777,-0.079970,0.064458,0.000250
164,0.118335,0.049550,0.133240,-0.159245,-0.237414,0.004430,0.960360,-0.194657,0.030932,-0.191618,...,0.185254,-0.316608,-0.030646,0.607250,0.108507,0.123180,-0.066611,0.151763,-0.162843,0.026763
165,0.112267,0.216307,0.156119,-0.369322,-0.217792,0.165465,0.994893,-0.383831,-0.050472,-0.008918,...,0.272632,-0.112599,-0.047830,0.439271,0.101899,0.213281,0.028534,0.023771,-0.207012,0.006931
166,0.072065,0.065555,0.134661,-0.128100,-0.149739,-0.045680,0.991981,-0.264321,-0.064478,-0.154887,...,0.232019,-0.136087,-0.125552,0.331902,0.136607,0.075867,-0.134309,-0.024143,-0.160347,0.059555


In [39]:
repvet_rotulado.head(3)

,0,1,2,3,4,5,6,7,8,9,...,759,760,761,762,763,764,765,766,767,rotulo
0,0.030631,0.149520,0.132294,-0.243208,-0.300681,0.193397,0.997896,-0.227564,-0.158239,-0.098940,...,-0.098749,-0.158954,0.123197,-0.038104,0.143516,0.056023,-0.026564,-0.212858,0.044322,0
1,0.121566,0.264066,0.046265,-0.087665,-0.114877,0.057351,0.914004,-0.251580,-0.094214,-0.042298,...,-0.007474,-0.124507,0.747115,0.076023,0.075091,0.006695,-0.047540,-0.213926,-0.121684,1
2,0.144653,0.043282,0.140579,-0.252127,-0.194191,0.112558,0.992621,-0.204062,-0.121806,0.061342,...,-0.152269,-0.139368,0.432061,0.078179,0.094881,-0.054676,-0.029966,-0.069750,-0.010176,0


# Criando uma estrutura para armazenar os resultados

In [40]:
# Algoritmos de classificação
classifiers = [  #NB, KNN, SVM
    ('Multinomial Naive Bayes', MultinomialNB()),
    ('Complement Naive Bayes Classifier', ComplementNB()),
    ('KNN', KNeighborsClassifier()),  #n_neighbors default é 5
    ('SVM', svm.SVC( )),
     ('Random Forest', RandomForestClassifier(random_state=42)),
      ('AdaBoost',    AdaBoostClassifier(random_state=42)) #n_estimators default é 50
]

In [41]:
lista_classificador_nome = list()
for classifier_name, classifier in classifiers:
    lista_classificador_nome.append(classifier_name)

In [42]:
df_acc = pd.DataFrame(columns=['Classificador','Rodada 1', 'Rodada 2', 'Rodada 3', 'Rodada 4', 'Rodada 5', 'Media'])
df_f1 = pd.DataFrame(columns=['Classificador','Rodada 1', 'Rodada 2', 'Rodada 3', 'Rodada 4', 'Rodada 5', 'Media'])
df_f1_ponderado = pd.DataFrame(columns=['Classificador','Rodada 1', 'Rodada 2', 'Rodada 3', 'Rodada 4', 'Rodada 5', 'Media'])

In [43]:
for classifier_name, classifier in classifiers:
    nova_linha = pd.DataFrame({'Classificador': [classifier_name], 'Rodada 1':[0] , 'Rodada 2':[0], 'Rodada 3':[0], 'Rodada 4':[0], 'Rodada 5':[0], 'Media':[0]})
    df_acc = pd.concat([df_acc, nova_linha], ignore_index=True)
    df_f1 = pd.concat([df_f1, nova_linha], ignore_index=True)
    df_f1_ponderado = pd.concat([df_f1_ponderado, nova_linha], ignore_index=True)

In [44]:
df_f1

,Classificador,Rodada 1,Rodada 2,Rodada 3,Rodada 4,Rodada 5,Media
0,Multinomial Naive Bayes,0,0,0,0,0,0
1,Complement Naive Bayes Classifier,0,0,0,0,0,0
2,KNN,0,0,0,0,0,0
3,SVM,0,0,0,0,0,0
4,Random Forest,0,0,0,0,0,0
5,AdaBoost,0,0,0,0,0,0


In [45]:
df_f1_ponderado

,Classificador,Rodada 1,Rodada 2,Rodada 3,Rodada 4,Rodada 5,Media
0,Multinomial Naive Bayes,0,0,0,0,0,0
1,Complement Naive Bayes Classifier,0,0,0,0,0,0
2,KNN,0,0,0,0,0,0
3,SVM,0,0,0,0,0,0
4,Random Forest,0,0,0,0,0,0
5,AdaBoost,0,0,0,0,0,0


In [46]:
df_acc

,Classificador,Rodada 1,Rodada 2,Rodada 3,Rodada 4,Rodada 5,Media
0,Multinomial Naive Bayes,0,0,0,0,0,0
1,Complement Naive Bayes Classifier,0,0,0,0,0,0
2,KNN,0,0,0,0,0,0
3,SVM,0,0,0,0,0,0
4,Random Forest,0,0,0,0,0,0
5,AdaBoost,0,0,0,0,0,0


 Usei como parâmetro para **average** o **'macro'** para o f1, e o **'weighted'** para o f1-ponderado

**'weighted'**:

Calculate metrics for each label, and find their average weighted by support (the number of true instances for each label). This alters ‘macro’ to account for label imbalance; it can result in an F-score that is not between precision and recall.

**'macro'**:

Calculate metrics for each label, and find their unweighted mean. This does not take label imbalance into account.

 vide [Documentação oficial](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_recall_fscore_support.html)

# Aplicando modelo de classificação

In [47]:
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score
from sklearn.metrics import precision_recall_fscore_support as score

def evaluate_model(model, X_test, y_test):
    # Predição dos rótulos
    y_pred = model.predict(X_test)
    print(classification_report(y_test, y_pred))

    # Cálculo da matriz de confusão
    cm = confusion_matrix(y_test, y_pred)

    # Cálculo da acurácia
    acc = accuracy_score(y_test, y_pred)

    # Cálculo do F1-score
    f1 = f1_score(y_test, y_pred, average='macro')

    # Cálculo do F1-score
    f1_poderado = f1_score(y_test, y_pred, average='weighted')

    # Outras métricas
    precision, recall, f1score, support = score(y_test, y_pred, average='macro')
    return cm, acc, f1, precision, recall, f1score, support, f1_poderado

In [48]:
df_repvet

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,0.030631,0.149520,0.132294,-0.243208,-0.300681,0.193397,0.997896,-0.227564,-0.158239,-0.098940,...,0.216850,-0.098749,-0.158954,0.123197,-0.038104,0.143516,0.056023,-0.026564,-0.212858,0.044322
1,0.121566,0.264066,0.046265,-0.087665,-0.114877,0.057351,0.914004,-0.251580,-0.094214,-0.042298,...,0.211784,-0.007474,-0.124507,0.747115,0.076023,0.075091,0.006695,-0.047540,-0.213926,-0.121684
2,0.144653,0.043282,0.140579,-0.252127,-0.194191,0.112558,0.992621,-0.204062,-0.121806,0.061342,...,0.231817,-0.152269,-0.139368,0.432061,0.078179,0.094881,-0.054676,-0.029966,-0.069750,-0.010176
3,0.168949,0.107046,-0.021332,-0.106200,-0.136551,-0.020100,0.955896,-0.237745,-0.073313,-0.012520,...,0.293984,-0.169405,-0.054496,0.572342,0.119459,0.050222,-0.172856,-0.062752,-0.100114,-0.045655
4,-0.026465,0.219727,0.280597,-0.274787,-0.095324,0.298268,0.670012,-0.059023,-0.145148,0.173758,...,0.186416,-0.133134,-0.068988,0.866958,0.174363,0.240498,0.040937,-0.179445,-0.036021,-0.071915
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,-0.020350,-0.066606,-0.028286,0.112232,-0.086299,0.011594,0.986838,0.002186,0.138783,-0.178709,...,0.114298,-0.268188,0.071848,0.485660,-0.101025,-0.046000,-0.141777,-0.079970,0.064458,0.000250
164,0.118335,0.049550,0.133240,-0.159245,-0.237414,0.004430,0.960360,-0.194657,0.030932,-0.191618,...,0.185254,-0.316608,-0.030646,0.607250,0.108507,0.123180,-0.066611,0.151763,-0.162843,0.026763
165,0.112267,0.216307,0.156119,-0.369322,-0.217792,0.165465,0.994893,-0.383831,-0.050472,-0.008918,...,0.272632,-0.112599,-0.047830,0.439271,0.101899,0.213281,0.028534,0.023771,-0.207012,0.006931
166,0.072065,0.065555,0.134661,-0.128100,-0.149739,-0.045680,0.991981,-0.264321,-0.064478,-0.154887,...,0.232019,-0.136087,-0.125552,0.331902,0.136607,0.075867,-0.134309,-0.024143,-0.160347,0.059555


- Alguns algoritmos só lidam com valores positivos, por isso precisamos normalizar os valores que aparecem na RV do BERT e GPT-2:

In [49]:
scaler = MinMaxScaler()
df_repvet_pos = scaler.fit_transform(df_repvet)
scaling_factor = 1000
df_repvet_pos_i = (df_repvet_pos * scaling_factor).astype(int)
df_repvet_pos_i = pd.DataFrame(df_repvet_pos_i, columns=df_repvet.columns)

In [50]:
df_repvet_pos_i.describe()

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
count,168.000000,168.000000,168.000000,168.000000,168.000000,168.000000,168.000000,168.000000,168.000000,168.000000,...,168.000000,168.000000,168.000000,168.000000,168.000000,168.000000,168.000000,168.000000,168.000000,168.000000
mean,526.351190,530.577381,474.196429,481.380952,498.517857,524.416667,968.214286,493.136905,416.339286,427.892857,...,590.130952,466.386905,475.238095,532.720238,547.214286,566.744048,513.970238,479.904762,507.315476,551.238095
std,200.415112,236.881372,199.423999,172.691786,188.308670,171.518593,113.173077,231.500855,208.404111,212.042788,...,157.968335,216.075334,209.608853,206.204652,206.935558,163.749041,172.952641,191.096058,201.402244,177.476598
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,401.750000,348.500000,333.750000,371.750000,375.000000,404.250000,981.000000,318.750000,279.500000,271.000000,...,487.750000,290.000000,341.500000,378.000000,408.750000,457.000000,392.500000,359.000000,367.750000,444.500000
50%,552.500000,530.500000,498.500000,469.500000,492.500000,514.500000,990.000000,452.500000,399.500000,419.000000,...,597.500000,475.000000,432.000000,552.500000,591.500000,548.000000,483.500000,474.500000,512.000000,541.500000
75%,682.000000,732.250000,630.750000,604.000000,624.000000,649.750000,995.000000,633.000000,518.750000,541.000000,...,687.750000,616.250000,587.250000,673.000000,685.250000,683.500000,641.250000,614.000000,644.000000,680.250000
max,1000.000000,1000.000000,1000.000000,1000.000000,999.000000,999.000000,1000.000000,1000.000000,1000.000000,1000.000000,...,1000.000000,1000.000000,1000.000000,999.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,999.000000


In [51]:
classifiers

[('Multinomial Naive Bayes', MultinomialNB()),
 ('Complement Naive Bayes Classifier', ComplementNB()),
 ('KNN', KNeighborsClassifier()),
 ('SVM', SVC()),
 ('Random Forest', RandomForestClassifier(random_state=42)),
 ('AdaBoost', AdaBoostClassifier(random_state=42))]

In [52]:
classificador=0
for classifier_name, classifier in classifiers:
    print('---', classifier_name, '---')
    y_true = []
    y_pred = []
    contador = 0
    serie_acc = pd.Series()
    serie_f1 = pd.Series()
    serie_f1_ponderado = pd.Series()

    #Substituindo o dataframe
    # if (classifier_name == 'Multinomial Naive Bayes' or classifier_name == "Complement Naive Bayes Classifier"):
    #     df_substituto = df_repvet_pos
    # else:
    #     df_substituto = df_repvet
    df_substituto= df_repvet_pos_i

    for i in range(0, 5):  #Estou percorrendo as 5 rodadas
        train_index = kfold_train[i]
        test_index = kfold_test[i]
        contador +=1
        X_train, X_test = df_substituto.iloc[train_index], df_substituto.iloc[test_index]
        y_train, y_test = repvet_rotulado.iloc[train_index]['rotulo'], repvet_rotulado.iloc[test_index]['rotulo']

        # Treinamento do modelo
        classifier.fit(X_train, y_train)
        cm, acc, f1, precision, recall, f1score, support,f1_poderado = evaluate_model(classifier, X_test, y_test)


        print(classifier_name + " Rodada " + str(contador) )
        print('Matriz de Confusão:')
        print(cm)
        print('Acurácia:', acc)
        print('F1-Score:', f1)
        print("outras métricas:")
        print('precision:', precision)
        print('recall:', recall)
        print('f1score:', f1score)
        print('support:', support)
        print('-------------------------------------')
        # serie_acc = serie_acc.append(pd.Series([acc]))
        serie_acc = pd.concat([serie_acc, pd.Series([acc])])
        # serie_f1 = serie_f1.append(pd.Series([f1]))
        serie_f1 = pd.concat([serie_f1, pd.Series([f1])])
        serie_f1_ponderado = pd.concat([serie_f1_ponderado, pd.Series([f1_poderado])])


    # Avaliação do modelo: Aqui estamos inserindo os valores das medias na serie
    media_acc = serie_acc[:5].mean()
    media_f1 = serie_f1[:5].mean()
    media_f1_ponderado = serie_f1_ponderado[:5].mean()
    # serie_acc = serie_acc.append(pd.Series([media_acc]))
    # serie_f1 = serie_f1.append(pd.Series([media_f1]))
    serie_acc = pd.concat([serie_acc, pd.Series([media_acc])])
    serie_f1 = pd.concat([serie_f1, pd.Series([media_f1])])
    serie_f1_ponderado = pd.concat([serie_f1_ponderado, pd.Series([media_f1_ponderado])])

    # print("Acurácia: " )
    # print(serie_acc)
    # print("F-1: " )
    # print(serie_f1)
    df_acc.loc[classificador, ['Rodada 1', 'Rodada 2', 'Rodada 3', 'Rodada 4', 'Rodada 5', 'Media']] = serie_acc.values
    df_f1.loc[classificador, ['Rodada 1', 'Rodada 2', 'Rodada 3', 'Rodada 4', 'Rodada 5', 'Media']] = serie_f1.values
    df_f1_ponderado.loc[classificador, ['Rodada 1', 'Rodada 2', 'Rodada 3', 'Rodada 4', 'Rodada 5', 'Media']] = serie_f1_ponderado.values
    classificador+=1
    print("=======================================================================================")
    # cm = confusion_matrix(y_true, y_pred)
    # acc = accuracy_score(y_true, y_pred)

--- Multinomial Naive Bayes ---
              precision    recall  f1-score   support

           0       0.76      0.68      0.72        19
           1       0.14      0.12      0.13         8
           2       0.70      1.00      0.82         7

    accuracy                           0.62        34
   macro avg       0.54      0.60      0.56        34
weighted avg       0.61      0.62      0.60        34

Multinomial Naive Bayes Rodada 1
Matriz de Confusão:
[[13  6  0]
 [ 4  1  3]
 [ 0  0  7]]
Acurácia: 0.6176470588235294
F1-Score: 0.5596949891067537
outras métricas:
precision: 0.5358543417366947
recall: 0.6030701754385964
f1score: 0.5596949891067537
support: None
-------------------------------------
              precision    recall  f1-score   support

           0       0.64      0.74      0.68        19
           1       0.00      0.00      0.00         8
           2       1.00      0.71      0.83         7

    accuracy                           0.56        34
   macro avg 

<ipython-input-52-ecc6b239afaf>:7: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  serie_acc = pd.Series()
<ipython-input-52-ecc6b239afaf>:8: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  serie_f1 = pd.Series()
<ipython-input-52-ecc6b239afaf>:9: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  serie_f1_ponderado = pd.Series()


              precision    recall  f1-score   support

           0       0.62      0.79      0.70        19
           1       0.00      0.00      0.00         8
           2       0.86      0.86      0.86         7

    accuracy                           0.62        34
   macro avg       0.49      0.55      0.52        34
weighted avg       0.53      0.62      0.57        34

Multinomial Naive Bayes Rodada 3
Matriz de Confusão:
[[15  3  1]
 [ 8  0  0]
 [ 1  0  6]]
Acurácia: 0.6176470588235294
F1-Score: 0.5182724252491694
outras métricas:
precision: 0.49404761904761907
recall: 0.5488721804511277
f1score: 0.5182724252491694
support: None
-------------------------------------
              precision    recall  f1-score   support

           0       0.82      0.95      0.88        19
           1       0.40      0.25      0.31         8
           2       0.67      0.67      0.67         6

    accuracy                           0.73        33
   macro avg       0.63      0.62      0.62 

<ipython-input-52-ecc6b239afaf>:7: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  serie_acc = pd.Series()
<ipython-input-52-ecc6b239afaf>:8: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  serie_f1 = pd.Series()
<ipython-input-52-ecc6b239afaf>:9: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  serie_f1_ponderado = pd.Series()
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib

              precision    recall  f1-score   support

           0       0.67      0.95      0.78        19
           1       0.00      0.00      0.00         8
           2       0.86      0.86      0.86         7

    accuracy                           0.71        34
   macro avg       0.51      0.60      0.55        34
weighted avg       0.55      0.71      0.61        34

Complement Naive Bayes Classifier Rodada 3
Matriz de Confusão:
[[18  0  1]
 [ 8  0  0]
 [ 1  0  6]]
Acurácia: 0.7058823529411765
F1-Score: 0.546583850931677
outras métricas:
precision: 0.5079365079365079
recall: 0.6015037593984962
f1score: 0.546583850931677
support: None
-------------------------------------
              precision    recall  f1-score   support

           0       0.75      0.95      0.84        19
           1       0.00      0.00      0.00         8
           2       0.44      0.67      0.53         6

    accuracy                           0.67        33
   macro avg       0.40      0.54    

...

              precision    recall  f1-score   support

           0       0.66      1.00      0.79        19
           1       0.00      0.00      0.00         8
           2       1.00      0.71      0.83         7

    accuracy                           0.71        34
   macro avg       0.55      0.57      0.54        34
weighted avg       0.57      0.71      0.61        34

Random Forest Rodada 2
Matriz de Confusão:
[[19  0  0]
 [ 8  0  0]
 [ 2  0  5]]
Acurácia: 0.7058823529411765
F1-Score: 0.5416666666666666
outras métricas:
precision: 0.5517241379310345
recall: 0.5714285714285715
f1score: 0.5416666666666666
support: None
-------------------------------------
              precision    recall  f1-score   support

           0       0.69      0.95      0.80        19
           1       0.50      0.12      0.20         8
           2       1.00      0.86      0.92         7

    accuracy                           0.74        34
   macro avg       0.73      0.64      0.64        34
w

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif

              precision    recall  f1-score   support

           0       0.70      1.00      0.83        19
           1       0.00      0.00      0.00         8
           2       0.67      0.67      0.67         6

    accuracy                           0.70        33
   macro avg       0.46      0.56      0.50        33
weighted avg       0.53      0.70      0.60        33

Random Forest Rodada 4
Matriz de Confusão:
[[19  0  0]
 [ 6  0  2]
 [ 2  0  4]]
Acurácia: 0.696969696969697
F1-Score: 0.4975845410628019
outras métricas:
precision: 0.4567901234567901
recall: 0.5555555555555555
f1score: 0.4975845410628019
support: None
-------------------------------------
              precision    recall  f1-score   support

           0       0.68      0.94      0.79        18
           1       0.50      0.11      0.18         9
           2       0.83      0.83      0.83         6

    accuracy                           0.70        33
   macro avg       0.67      0.63      0.60        33
we

<ipython-input-52-ecc6b239afaf>:7: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  serie_acc = pd.Series()
<ipython-input-52-ecc6b239afaf>:8: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  serie_f1 = pd.Series()
<ipython-input-52-ecc6b239afaf>:9: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  serie_f1_ponderado = pd.Series()


              precision    recall  f1-score   support

           0       0.74      0.74      0.74        19
           1       0.18      0.25      0.21         8
           2       0.75      0.43      0.55         7

    accuracy                           0.56        34
   macro avg       0.56      0.47      0.50        34
weighted avg       0.61      0.56      0.57        34

AdaBoost Rodada 1
Matriz de Confusão:
[[14  5  0]
 [ 5  2  1]
 [ 0  4  3]]
Acurácia: 0.5588235294117647
F1-Score: 0.4976076555023923
outras métricas:
precision: 0.5562200956937798
recall: 0.47180451127819545
f1score: 0.4976076555023923
support: None
-------------------------------------
              precision    recall  f1-score   support

           0       0.64      0.47      0.55        19
           1       0.17      0.38      0.23         8
           2       1.00      0.29      0.44         7

    accuracy                           0.41        34
   macro avg       0.60      0.38      0.41        34
weigh

In [53]:
df_acc

,Classificador,Rodada 1,Rodada 2,Rodada 3,Rodada 4,Rodada 5,Media
0,Multinomial Naive Bayes,0.617647,0.558824,0.617647,0.727273,0.606061,0.62549
1,Complement Naive Bayes Classifier,0.764706,0.647059,0.705882,0.666667,0.636364,0.684135
2,KNN,0.794118,0.735294,0.705882,0.69697,0.606061,0.707665
3,SVM,0.735294,0.705882,0.735294,0.69697,0.69697,0.714082
4,Random Forest,0.705882,0.705882,0.735294,0.69697,0.69697,0.7082
5,AdaBoost,0.558824,0.411765,0.647059,0.757576,0.606061,0.596257


In [54]:
df_f1

,Classificador,Rodada 1,Rodada 2,Rodada 3,Rodada 4,Rodada 5,Media
0,Multinomial Naive Bayes,0.559695,0.50542,0.518272,0.617469,0.561538,0.552479
1,Complement Naive Bayes Classifier,0.560847,0.484472,0.546584,0.456848,0.47619,0.504988
2,KNN,0.650171,0.641026,0.546584,0.55193,0.55873,0.589688
3,SVM,0.560224,0.547281,0.577196,0.497585,0.544444,0.545346
4,Random Forest,0.550933,0.541667,0.641026,0.497585,0.60195,0.566632
5,AdaBoost,0.497608,0.406889,0.596761,0.655428,0.574603,0.546258


In [55]:
df_f1_ponderado

,Classificador,Rodada 1,Rodada 2,Rodada 3,Rodada 4,Rodada 5,Media
0,Multinomial Naive Bayes,0.604517,0.553204,0.566347,0.701347,0.579487,0.600981
1,Complement Naive Bayes Classifier,0.665733,0.560102,0.613811,0.578999,0.536797,0.591088
2,KNN,0.72744,0.684163,0.613811,0.636835,0.572294,0.646909
3,SVM,0.648542,0.623383,0.64186,0.596838,0.587879,0.6197
4,Random Forest,0.632965,0.613971,0.684163,0.596838,0.632392,0.632066
5,AdaBoost,0.5736,0.450615,0.652465,0.739055,0.618182,0.606783
